### Import and initialize

In [39]:
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
import torch
from tqdm import tqdm
from torch.utils.data import TensorDataset, random_split
DEVICE = torch.device('cpu')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
CHECKPOINT_PATH = '/home/student/workspace/Truthseeker/checkpoints/checkpoint_with_maxlength_400'

## Loading the model

In [40]:
model = BertForSequenceClassification.from_pretrained(
    CHECKPOINT_PATH, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

#Loading from statedict
#model.load_state_dict(torch.load('final.ckpt'))

# Tell pytorch to run this model on the GPU.
model.to(DEVICE)
model = model.eval()

## Loading the dataset

In [41]:
import pandas as pd
DATASET_PATH = "/home/student/datasets/TruthSeeker2023/Truth_Seeker_Model_Dataset.csv"
df = pd.read_csv(DATASET_PATH)

print('Number of training sentences: {:,}\n'.format(df.shape[0]))
#Shuffling the dataset
df = df.sample(frac=1)

Number of training sentences: 134,198



In [42]:
sentences = 'Statement: ' + df['statement'] + '| Tweet: ' +df['tweet']
labels = df["BinaryNumTarget"].values

## Tokenize the dataset

In [43]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []
MAX_SENTENCE_LENGTH = 410

# For every sentence...
for sent in tqdm(sentences[:1000]):
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = MAX_SENTENCE_LENGTH,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

  0%|                                          | 0/1000 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/student/miniconda3/envs/truth_seeker/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2606: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.wa

In [44]:
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)
# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels[:1000])

## Inference any single data point

In [45]:
INDEX = 235
with torch.no_grad():
    #output = model(dataset[INDEX:INDEX + 2][1], token_type_ids=None, attention_mask=dataset[INDEX:INDEX + 2][1],labels=None)
    b_input_ids = dataset[INDEX:INDEX + 1][0]
    attention_mask = dataset[INDEX:INDEX + 1][1]
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        
        output = model(b_input_ids, 
                               token_type_ids=None, 
                               attention_mask=attention_mask,
                               labels=None)
        loss = output.loss
        logits = output.logits
    print ("Prediction", torch.argmax(logits), "| Label:", dataset[INDEX: INDEX+ 1][2])

Prediction tensor(0) | Label: tensor([0.], dtype=torch.float64)
